In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import torch
import os 
from models.utils import make_mp4_from_imgs, show_mp4, load_model, save_image, normal_samples
from models.utils import plot_losses_and_save_plot, make_gif_from_imgs, show_gif
%matplotlib qt

In [ ]:
if not os.path.exists('best_images'):
    os.mkdir('best_images')
else:
    for f in os.listdir('best_images'):
        os.remove(os.path.join('best_images', f))
    os.rmdir('best_images')
    os.mkdir('best_images')

In [3]:
import re


normal_sample = normal_samples.sample(torch.Size([25])).to(torch.device('cuda'))
i=0
filenames = sorted(os.listdir('saved_models'), key=lambda x: int(''.join(re.findall(r'\d+', x))))


In [ ]:
np.save('filenames_sorted.npy', filenames)
for model_file in filenames:
    model = load_model(os.path.join('saved_models', model_file))
    model.eval()
    model=model.to(torch.device('cuda'))
    output = model(normal_sample)
    save_image(((output.cpu().detach() + 1) / 2).unsqueeze(1), 'best_images' +
                       "/%5d.png" % i, nrow=5, normalize=False)
    i+=1

 

In [ ]:
make_mp4_from_imgs('best_images', 'best_images.mp4', n=1)
show_mp4('best_images.mp4')

In [ ]:
#make_gif_from_imgs("images", 50, 0, 'training_images.gif')

In [ ]:
#show_gif('training_images.gif')

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = load_model(os.path.join('saved_models', filenames[5*60+33]))
model.eval()
model = model.to(device)

In [14]:
import shutil
from models.utils import save_model

# Move a file from one location to another
file = os.path.join('saved_models', filenames[5*60+33])
new_file = 'best_model_.th'
shutil.copyfile(file, new_file)


'best_model_epoch_333.th'

In [5]:
if not os.path.exists('eval_images'):
    os.mkdir('eval_images')
else:
    for f in os.listdir('eval_images'):
        os.remove(os.path.join('eval_images', f))
    os.rmdir('eval_images')
    os.mkdir('eval_images')

In [10]:

for i in range(300):
    z = normal_samples.sample(torch.Size([25])) * 0.8
    z = z.to(device)
    imgs = model(z).cpu().detach() * 0.5 + 0.5
    if len(imgs.shape) == 3:
        imgs = imgs.unsqueeze(1)
    
    save_image(imgs, 'eval_images' +
                       "/%5d.png" % i, nrow=5, normalize=False)


In [11]:
make_gif_from_imgs('eval_images', 650, 0, 'eval_images.gif')
show_gif('eval_images.gif')

In [36]:
d_losses, g_losses = np.load('d_losses.npy'), np.load('g_losses.npy')
fig, ax = plt.subplots(figsize=(12, 8))
ax.plot(g_losses, label='Generator Loss', color='orange')
ax.plot(d_losses, label='Discriminator Loss', color='darkgreen')
ax.set_xlabel('Epoch')
ax.set_ylabel('Loss')
ax.set_title('GAN Loss')
ax.legend()
plt.tight_layout()
plt.savefig('losses.png')
plt.show()
